In [17]:
from KG_evaluator import KGEvaluator
from ontology.loader import PEKGOntology
import networkx as nx

In [19]:
kg_path = "C:/PE/outputs/AIRELLE/knowledge_graph_AIRELLE_gpt-4.1_iterative.json"
ontology_path = "C:/PE/REPOS/llm_kg_extraction/llm_kg_extraction/ontology/pekg_ontology.yaml"

kge = KGEvaluator(kg_path, ontology_path)

In [20]:
metrics = kge.basic_metrics()

In [21]:
quality = kge.evaluate_graph_quality()

In [22]:
largest_cc = max(nx.weakly_connected_components(kge.graph), key=len)
subgraph = kge.graph.subgraph(largest_cc)

In [23]:
nx.diameter(subgraph.to_undirected())

5

In [24]:
metrics

{'num_nodes': 267,
 'num_edges': 328,
 'density': 0.004618287291261862,
 'avg_degree': 2.4569288389513106,
 'connected_components': 6,
 'node_type_distribution': {'Company': 20,
  'Product': 54,
  'UseCase': 55,
  'CorporateEvent': 4,
  'KPI': 42,
  'GovernmentBody': 5,
  'Contract': 2,
  'RegulatoryRequirement': 2,
  'FinancialMetric': 8,
  'MarketMetric': 35,
  'Department': 13,
  'Location': 16,
  'HeadcountMetric': 3,
  'Person': 4,
  'Position': 4}}

In [25]:
quality

{'isolated_nodes_ratio': 0.0,
 'attribute_completeness': 1.0,
 'financial_metrics_availability': False,
 'avg_company_relations': 11.2,
 'graph_diameter': 5}

In [26]:
distribution_relations = kge.relation_distribution()
distribution_entities = kge.entity_coverage()

In [27]:
distribution_relations

({'ownsEntity': 0.036585365853658534,
  'hasCustomer': 0.07317073170731707,
  'hasHeadcount': 0.01524390243902439,
  'operatesInMarket': 0.06097560975609756,
  'hasEvent': 0.012195121951219513,
  'signsContract': 0.012195121951219513,
  'hasKPI': 0.18597560975609756,
  'subjectTo': 0.012195121951219513,
  'reportsMetric': 0.1402439024390244,
  'targetsUseCase': 0.31402439024390244,
  'mentionsCompany': 0.003048780487804878,
  'operatesInLocation': 0.027439024390243903,
  'hasHeadquartersIn': 0.006097560975609756,
  'hasDepartment': 0.03353658536585366,
  'hasContractor': 0.009146341463414634,
  'hasOfficeIn': 0.024390243902439025,
  'hasPartnershipWith': 0.003048780487804878,
  'hasProduct': 0.003048780487804878,
  'contractsWithGov': 0.003048780487804878,
  'employs': 0.012195121951219513,
  'holdsPosition': 0.012195121951219513},
 {'ownsEntity': 12,
  'hasCustomer': 24,
  'hasHeadcount': 5,
  'operatesInMarket': 20,
  'hasEvent': 4,
  'signsContract': 4,
  'hasKPI': 61,
  'subjectTo'

In [28]:
distribution_entities[1]

defaultdict(int,
            {'Company': 20,
             'Product': 54,
             'UseCase': 55,
             'CorporateEvent': 4,
             'KPI': 42,
             'GovernmentBody': 5,
             'Contract': 2,
             'RegulatoryRequirement': 2,
             'FinancialMetric': 8,
             'MarketMetric': 35,
             'Department': 13,
             'Location': 16,
             'HeadcountMetric': 3,
             'Person': 4,
             'Position': 4,
             'LegalEntity': 0,
             'Committee': 0,
             'Investor': 0,
             'Advisor': 0,
             'FinancialInstrument': 0,
             'OwnershipStake': 0,
             'IntellectualProperty': 0,
             'Litigation': 0,
             'Risk': 0,
             'PolicyDocument': 0,
             'FundingRound': 0,
             'AcquisitionEvent': 0,
             'MergerEvent': 0,
             'IPOEvent': 0,
             'ExitEvent': 0,
             'LeadershipChangeEvent': 0,
        

In [29]:
quality != kge.evaluate_graph_quality()
quality

{'isolated_nodes_ratio': 0.0,
 'attribute_completeness': 1.0,
 'financial_metrics_availability': False,
 'avg_company_relations': 11.2,
 'graph_diameter': 5}

In [30]:
ontology_compliance = kge.evaluate_ontology_compliance()
ontology_compliance

{'entity_type_validity': 1.0,
 'relation_type_validity': 0.9939024390243902,
 'domain_range_validity': 0.3048780487804878,
 'attribute_validity': 1.0,
 'overall_compliance': 0.8246951219512195,
 'validation_details': {'invalid_entities': [],
  'invalid_relations': [{'source': 'e220',
    'target': 'e248',
    'invalid_type': 'hasPartnershipWith'},
   {'source': 'e220', 'target': 'e254', 'invalid_type': 'hasProduct'}],
  'domain_range_violations': [{'relation_type': 'ownsEntity',
    'source': {'id': 'e1', 'type': 'Company'},
    'target': {'id': 'e2', 'type': 'Company'},
    'expected_domain': ['Company'],
    'expected_range': ['LegalEntity']},
   {'relation_type': 'hasCustomer',
    'source': {'id': 'e1', 'type': 'Company'},
    'target': {'id': 'e12', 'type': 'Product'},
    'expected_domain': ['Company'],
    'expected_range': ['Company']},
   {'relation_type': 'hasHeadcount',
    'source': {'id': 'e1', 'type': 'Company'},
    'target': {'id': 'e13', 'type': 'Product'},
    'expect

In [31]:
kge.evaluate()

{'basic_metrics': {'num_nodes': 267,
  'num_edges': 328,
  'density': 0.004618287291261862,
  'avg_degree': 2.4569288389513106,
  'connected_components': 6,
  'node_type_distribution': {'Company': 20,
   'Product': 54,
   'UseCase': 55,
   'CorporateEvent': 4,
   'KPI': 42,
   'GovernmentBody': 5,
   'Contract': 2,
   'RegulatoryRequirement': 2,
   'FinancialMetric': 8,
   'MarketMetric': 35,
   'Department': 13,
   'Location': 16,
   'HeadcountMetric': 3,
   'Person': 4,
   'Position': 4}},
 'ontology_compliance': {'entity_type_validity': 1.0,
  'relation_type_validity': 0.9939024390243902,
  'domain_range_validity': 0.3048780487804878,
  'attribute_validity': 1.0,
  'overall_compliance': 0.8246951219512195,
  'validation_details': {'invalid_entities': [],
   'invalid_relations': [{'source': 'e220',
     'target': 'e248',
     'invalid_type': 'hasPartnershipWith'},
    {'source': 'e220', 'target': 'e254', 'invalid_type': 'hasProduct'}],
   'domain_range_violations': [{'relation_type':

In [7]:
kge.evaluate()

{'basic_metrics': {'num_nodes': 47,
  'num_edges': 47,
  'density': 0.021739130434782608,
  'avg_degree': 2.0,
  'connected_components': 1,
  'node_type_distribution': {'Company': 22,
   'Product': 1,
   'UseCase': 3,
   'GovernmentBody': 7,
   'FinancialMetric': 2,
   'HeadcountMetric': 1,
   'Person': 1,
   'LegalEntity': 1,
   'Position': 1,
   'Committee': 1,
   'Location': 4,
   'Contract': 1,
   'CorporateEvent': 1,
   'NewsItem': 1}},
 'ontology_compliance': {'entity_type_validity': 1.0,
  'relation_type_validity': 0.9148936170212766,
  'domain_range_validity': 0.7659574468085106,
  'attribute_validity': 1.0,
  'overall_compliance': 0.9202127659574468,
  'validation_details': {'invalid_entities': [],
   'invalid_relations': [{'source': 'e1',
     'target': 'e2',
     'invalid_type': 'hasProduct'},
    {'source': 'e1', 'target': 'e30', 'invalid_type': 'competesWith'},
    {'source': 'e1', 'target': 'e31', 'invalid_type': 'competesWith'},
    {'source': 'e45', 'target': 'e35', 'in